In [70]:
docs = [
    "resume/Aman_Kumar_Resume_entry_lvl_mld.pdf",
    "resume/resume (1).pdf",
    "resume/resume (2).pdf",
    "resume/resume (3).pdf",
    "resume/resume (4).pdf",
    "resume/resume (5).pdf",
    "resume/resume (6).pdf",
    "resume/resume (7).pdf",
]

# !pip install -U spacy
# !python -m spacy download en_core_web_sm
# %pip install --upgrade pip


In [71]:
import fitz
import spacy
nlp = spacy.load("en_core_web_sm")

ModuleNotFoundError: No module named 'spacy'

In [ ]:
def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ''
    links = []
    for page in doc:
        text += page.get_text()
        for url in page.get_links():
            links.append(url['uri'])
    return text, links
        

In [ ]:
rawData = extract_text_from_pdf('Aman_Kumar_Resume_entry_lvl_mld.pdf')
rawText = rawData[0]
rawLinks = rawData[1]

In [ ]:
def extract_candidate_name():
    

In [55]:
print(rawText)

AMAN KUMAR
Machine Learning Engineer — Backend Systems
amansahu.er@gmail.com · +91-8319151766 · LinkedIn · GitHub
PROFESSIONAL SUMMARY
Software Engineer with 2+ years of experience building scalable backend systems and integrating machine learning
features into production applications. Hands-on experience in deploying Python-based ML services, designing REST APIs,
and managing end-to-end document and image processing workflows. Strong backend foundation with growing expertise
in applied machine learning.
TECHNICAL SKILLS
• Machine Learning: Model inference, basic NLP workflows, image processing, Scikit-learn, PyTorch (fundamentals)
• Backend & Systems: REST API design, Flask, FastAPI, Laravel, system integration
• Programming: Python, PHP, SQL, JavaScript
• Tools: Docker, Git, Linux, Postman
PROFESSIONAL EXPERIENCE
Software Engineer
Jul 2024 – Present
ISKCON Bangalore
Bangalore, India
• Developed and maintained backend services supporting 50k+ users, focusing on reliability and data co